In [2]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import requests
import math
import folium
from folium.plugins import MarkerCluster
import webbrowser
import time

CSV_PATH = "philly_adoptables_quick.csv"
USER_ZIP = "19122"
TOP_N = 15


def geocode_zip(zip_code, country="US"):
    url = "https://nominatim.openstreetmap.org/search"
    params = {"postalcode": zip_code, "country": country, "format": "json", "limit": 1}
    headers = {"User-Agent": "petfinder-zip-recommender/1.0"}
    if GEOCODER_EMAIL:
        params["email"] = GEOCODER_EMAIL
    r = requests.get(url, params=params, headers=headers, timeout=20)
    r.raise_for_status()
    data = r.json()
    if not data:
        return None, None
    lat = float(data[0]["lat"]); lon = float(data[0]["lon"])
    time.sleep(1.0)
    return lat, lon

def haversine_miles(lat1, lon1, lat2, lon2):
    R_km = 6371.0088
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlmb/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    km = R_km * c
    miles = km * 0.621371
    return miles

def build_address(row):
    parts = []
    for col in ["org_address1", "org_address2", "org_city", "org_state", "org_postcode", "org_country"]:
        v = row.get(col)
        if pd.notna(v) and str(v).strip():
            parts.append(str(v).strip())
    return ", ".join(parts) if parts else None

df = pd.read_csv(CSV_PATH)

df = df.copy()
if "org_latitude" not in df.columns or "org_longitude" not in df.columns:
    raise ValueError("CSV must contain org_latitude and org_longitude columns.")

if FILTER_TYPE:
    df = df[df["type"].fillna("").str.lower() == FILTER_TYPE.lower()]
if FILTER_STATUS and "status" in df.columns:
    df = df[df["status"].fillna("").str.lower() == FILTER_STATUS.lower()]

df = df[(df["org_latitude"].notna()) & (df["org_longitude"].notna())]
df = df.reset_index(drop=True)

if df.empty:
    raise ValueError("No available data after filtering.")

user_lat, user_lon = geocode_zip(USER_ZIP)
if user_lat is None:
    raise ValueError(f"ZIP {USER_ZIP} geocoding failed.")

def compute_distance(row):
    try:
        return haversine_miles(float(row["org_latitude"]), float(row["org_longitude"]), user_lat, user_lon)
    except Exception:
        return float("inf")

df["distance_miles"] = df.apply(compute_distance, axis=1)
df_sorted = df.sort_values("distance_miles").head(TOP_N).reset_index(drop=True)

cols_show = ["id","type","name","breed_primary","organization_name","org_city","org_state","org_postcode","distance_miles","url"]
print(df_sorted[cols_show].to_string(index=False))

m = folium.Map(location=[user_lat, user_lon], zoom_start=11, control_scale=True)
folium.Marker(
    [user_lat, user_lon],
    tooltip=f"Your ZIP: {USER_ZIP}",
    popup=f"<b>Your ZIP</b>: {USER_ZIP}<br>({user_lat:.5f}, {user_lon:.5f})",
    icon=folium.Icon(color="red", icon="home")
).add_to(m)

cluster = MarkerCluster(name=f"Nearest {TOP_N} animals").add_to(m)

for _, r in df_sorted.iterrows():
    lat, lon = float(r["org_latitude"]), float(r["org_longitude"])
    name = r.get("name", "")
    typ = r.get("type", "")
    breed = r.get("breed_primary", "")
    shelter = r.get("organization_name", "")
    city = r.get("org_city", "")
    state = r.get("org_state", "")
    zipc = r.get("org_postcode", "")
    dist = r.get("distance_miles", float("nan"))
    url = r.get("url", "")
    photo = r.get("photo_url", "")

    img_html = f'<img src="{photo}" style="width:180px;height:auto;border-radius:8px;margin-bottom:6px;" />' if pd.notna(photo) and str(photo).startswith("http") else ""
    popup_html = f"""
    <div style="font-family:Arial; font-size:12px;">
        {img_html}
        <b>{name}</b> ({typ}) – {breed if pd.notna(breed) else ""}
        <br><b>Shelter:</b> {shelter if pd.notna(shelter) else ""}
        <br><b>Location:</b> {city}, {state} {zipc}
        <br><b>Distance:</b> {dist:.1f} miles
        <br><a href="{url}" target="_blank">View on Petfinder</a>
    </div>
    """
    folium.Marker(
        [lat, lon],
        tooltip=f"{name} • {dist:.1f} mi",
        popup=folium.Popup(popup_html, max_width=260)
    ).add_to(cluster)

folium.LayerControl().add_to(m)
out_html = f"nearest_{TOP_N}_from_{USER_ZIP}.html"
m.save(out_html)
print(f"\nSaved map: {out_html}")
try:
    webbrowser.open(out_html)
except:
    pass

      id type                name       breed_primary               organization_name     org_city org_state  org_postcode  distance_miles                                                                                                                                                                                                                                              url
78470735  Dog            Mr Ziggy   Yorkshire Terrier Hindes Animal Rescue Team, Inc.   GLEN MILLS        PA         19342       18.835628      https://www.petfinder.com/dog/mr-ziggy-78470735/pa/glen-mills/hindes-animal-rescue-team-inc-pa1229/?referrer_id=315dc131-351b-42eb-bb78-ce5bd15ee5d4&utm_source=api&utm_medium=partnership&utm_content=315dc131-351b-42eb-bb78-ce5bd15ee5d4
78470779  Cat Sweetie aka Savanah Domestic Short Hair          Brandywine Valley SPCA West Chester        PA         19380       23.927874 https://www.petfinder.com/cat/sweetie-aka-savanah-78470779/pa/west-chester/brandywine-valley-spca-pa1